In [2]:
path1 = "C:\\Users\\cdchang\\Desktop\\gensim-2\\chinsese_dict.dict"
path2 = "C:\\Users\\cdchang\\Desktop\\gensim-2\\lda.model"

In [3]:
#genesim
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import STOPWORDS
import nltk
import numpy as np

class Gensimembedder(object):
    def __init__(self, **kwargs):
        self.trained = True
        if kwargs != None:
            if 'dictionary' not in kwargs.keys():
                self.dictionary = None
                self.trained = False
            else:
                self.dictionary = kwargs['dictionary']

            if 'model' not in kwargs.keys():
                self.model = None
                self.trained = False
            else:
                self.model = kwargs['model']

    def tokenize_(self, text):
        '''
        tokenize the text using the utils from Gensim

        Input:
            text(str) : text that you want to tokenize
        Output
            (list): tokenized text
        '''
        return [token for token in simple_preprocess(text) if token not in STOPWORDS]

    def embed(self, text):
        '''
        embed the raw text into the vector with the length of topic number

        Input:
            text(str) : text that you want to embed
        Output:
            (list): embedded text
        '''
        if self.trained:
            model = self.model
            dictionary = self.dictionary
            text = self.tokenize_(text)
            bow = self.dictionary.doc2bow(text)
            kindex = self.model.get_document_topics(bow)
            out = [0.] * self.model.num_topics
            for i, p in kindex:
                out[i] = p
            return np.array(out)
        else:
            print ('Load LDA model and dictionary')
            return None


    def embed_sent(self, text):
        '''
        tokenize the text by sentences first, then
        embed the raw text into the vector with the length of topic number

        Input:
            text(str) : text that you want to embed
        Output:
            (list): embedded text
        '''
        if self.trained:
            model = self.model
            dictionary = self.dictionary
            out = np.array([0.] * self.model.num_topics)
            sentences = len(nltk.sent_tokenize(text))
            for text in nltk.sent_tokenize(text):
                out += self.embed(text)
            return (out/sentences)
        else:
            print ('Load LDA model and dictionary')
            return None


if __name__ == '__main__':
    dictionary = corpora.Dictionary.load(path1)
    model =  models.LdaModel.load(path2)
    model = Gensimembedder(model = model, dictionary = dictionary)
    sample = "This review is based upon consistency of flavor and great customer service.\
      We came and there was an unknown issue that required a 25 minute wait for food.  \
      The employee notified us, and although hesitant, we decided to stay.  \
      We have been here numerous times before in the past years so we are familiar with this location.  \
      The employee was apologetic and gave us a free drink.  \
      That was a simple gesture but rarely do you see decent customer service anymore.  \
      We received our food and had an issue with an incorrect order.  \
      It was explained and the issue was resolved quickly.  They gave us a free appetizer.  \
      We do not expect perfection, nor free food.  \
      This restaurant cares for customers and works to provide a positive experience.  \
      We would return again because they have good food and they care.  \
      That is a rarity in today's restaurant culture.  Kudos to the manager for creating this culture. \
      Ordered- fried rive and Tofu, edamame, won ton soup, dynamite chx, and Thai curry."
    print (model.embed_sent(sample))

C:\Users\cdchang\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[ 0.          0.          0.05564236  0.          0.07673649  0.03076934
  0.          0.          0.          0.0534375   0.          0.          0.
  0.          0.          0.          0.0534375   0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.0276942
  0.03714567  0.          0.01646121  0.          0.          0.          0.
  0.          0.          0.          0.          0.03419218  0.          0.
  0.          0.          0.          0.          0.          0.17922175
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.13614636
  0.          0.          0.          0.          0.          0.          0.
  0.          0.

In [25]:
import json
import pandas as pd

def json_df(datapass):
    '''
    Load the json file and parse the file to pandas dataframe format
    
    Input:
        datapass(str) : directory to the json file
    Output:
        df(dataframe) : pandas dataframe object
    '''
    
    data = [] 
    with open(datapass, 'r', encoding='utf-8') as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

#business and review json files --> dataframes

DATAPASS1 = 'C:\\Users\\cdchang\\Downloads\\yelp_dataset\\dataset\\business.json'
DATAPASS2 = 'C:\\Users\\cdchang\\Downloads\\yelp_dataset\\dataset\\review.json'

#business = json_df(DATAPASS1)
#reviews = json_df(DATAPASS2)

#generate a genesim review embedding for a given business
def df_allcomp(b_id):
    subset = reviews.loc[reviews['business_id']==b_id]
    date = subset.loc[:,'date']
    text = subset.loc[:,'text']
    stars = subset.loc[:,'stars']

    df = pd.concat([date,text,stars],axis=1, join='inner')
    return df

def business_embeddings(b_id, reviews):
    dataframe = pd.DataFrame()
    df = df_allcomp(b_id)
    dictionary = corpora.Dictionary.load(path1)
    model =  models.LdaModel.load(path2)
    model = Gensimembedder(model = model, dictionary = dictionary)
    for t in df.loc[:,'text']:
        a = np.array(model.embed_sent(t))
        #df_temp = pd.DataFrame(model.embed_sent(t))
        
        np.vstack(a,)
        #print(df_temp)
        
        #dataframe = pd.concat([dataframe,df_temp],axis=1, join='inner')
        #print (model.embed_sent(t))
    print(dataframe)

TypeError: cannot concatenate a non-NDFrame object